<a href="https://colab.research.google.com/github/analyticswithadam/App_Script/blob/main/Gemini_Function_Calling_(Intro).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
import os
from google import genai
from google.genai import types
from google.genai.types import GenerateContentConfig, ThinkingConfig
from google.cloud import bigquery

import textwrap
from typing import Dict
from datetime import datetime
import json
from IPython.display import Markdown, display

from google.colab import userdata
cloud_project = userdata.get('Project')

In [ ]:
# Schema Function

bq_client = bigquery.Client(project=cloud_project)
table = bq_client.get_table('bigquery-public-data.austin_bikeshare.bikeshare_trips')
schema = {}
for field in table.schema:
  schema[field.name] = ('Type:' +field.field_type+', Description: '+field.description)
print(json.dumps(schema, indent=4))

In [ ]:
# Schema Function AI

def get_bigquery_schema(table_name: str) -> dict[str, str]:
    """
    Reads the schema from a BigQuery table.
    You should receive the full big query qualified table name if not make best guess.

    Args:
        table_name: The fully qualified table name (e.g., 'project.dataset.table').

    Returns:
        A dictionary representing the table schema.
        fieldname:Type:Description

    """
    bq_client = bigquery.Client(project=cloud_project)
    table = bq_client.get_table(table_name)
    schema = {}
    for field in table.schema:
        schema[field.name] = ('Type:' +field.field_type+', Description: '+field.description)
    return schema

In [ ]:
# Query Function

bq_client = bigquery.Client(project=cloud_project)
query = """
SELECT
  EXTRACT(YEAR FROM start_time) AS year,
  EXTRACT(QUARTER FROM start_time) AS quarter,
  COUNT(*) AS total_trips
FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE EXTRACT(YEAR FROM start_time) IN (2018, 2019, 2020)
GROUP BY
  year, quarter
ORDER BY
  total_trips DESC
"""
query_job = bq_client.query(query)
results = query_job.result()
print(results.to_dataframe())

In [ ]:
# Query Function AI

def run_bigquery_query(query: str) -> str:
    """
    Executes a BigQuery SQL query and returns results as a JSON string.

    Args:
        query: The BigQuery SQL query string to execute.

    Returns:
        A JSON formatted string containing the query results. Each row is represented
        as a dictionary, with column names as keys. Datetime objects are converted
        to ISO 8601 formatted strings.
    """
    # Big Query Code
    bq_client = bigquery.Client(project=cloud_project)
    query_job = bq_client.query(query)
    results = query_job.result()

    # Convert results to a list of dictionaries
    data = []
    for row in results:
        row_dict = dict(row)
        # Convert datetime objects to strings before adding to the dictionary
        for key, value in row_dict.items():
            if isinstance(value, datetime):
                row_dict[key] = value.isoformat()
        data.append(row_dict)

    # Return the results as a JSON string
    return json.dumps(data)


In [ ]:


# Create a client
client = genai.Client(
     vertexai=True,
     project=cloud_project,
     location="us-central1"
)


MODEL_ID = "gemini-2.5-flash-preview-05-20"

sys_prompt = """You are an expert data analyst
    I will give you a google big query table and query instructions
    Use the get_bigquery_schema tool to read the schema of the table and formulate the query to run.
    Then run a query or querries using the run_bigquery_query tool to get the data to answer the users questions
    """
format_prompt ="""
    Output Format Instructions:

    Please structure your response in the following four distinct sections, using the exact headings provided below.

    1. Query Results

    [Present the direct results of the query here in a clean, readable format (e.g., a table, a list, or a code block as appropriate).]

    2. SQL Query Used

    [Provide the exact SQL query or queries that were used to generate the results. Below the query, include a brief explanation of what the query does, breaking down the key clauses (SELECT, FROM, WHERE, etc.) and their purpose.]

    3. Explanation

    [Summarize the query results in clear, natural language. Explain what the data means and any significant patterns or conclusions that can be drawn from it.]

    4. Functions Used

    [List the sequence of functions or tools you used to generate this response. For each function, specify how many times it was called. For example:

    Google Search: 1 time
    code_interpreter: 2 times]
    """

# Generate a response with function calling
def call_llm(user_prompt):
  output = client.models.generate_content(
      model=MODEL_ID,
      contents=sys_prompt +" "+user_prompt+" "+format_prompt,
      config=types.GenerateContentConfig(
          tools=[run_bigquery_query, get_bigquery_schema],
          thinking_config=ThinkingConfig(include_thoughts=True)
      ),
  )
  return output


In [ ]:
# Print Response
response = call_llm("""Query the table bigquery-public-data.austin_bikeshare.bikeshare_trips \
 to get the most popular season for total trips in 2018, 2019 and 2020""")
display(Markdown(response.text))


In [ ]:
response = call_llm("""From the table bigquery-public-data.austin_311.311_service_requests \
 what were the most common complaint descriptions created in the last week of May 2025""")
display(Markdown(response.text))

In [ ]:
# Print Thoughts
print("Thoughts:")
for ci, candidate in enumerate(response.candidates, start=0):
    for part in candidate.content.parts:
        if getattr(part, "thought", False):
            wrapped = textwrap.fill(part.text, width=80)
            print("\nnew thought:")
            print(wrapped)


In [ ]:
# Calculate and print individual costs
thought_tokens = response.usage_metadata.thoughts_token_count
thought_cost = thought_tokens * (3.50 / 1_000_000)

output_tokens = response.usage_metadata.candidates_token_count
output_cost = output_tokens * (0.60 / 1_000_000)

input_tokens = response.usage_metadata.prompt_token_count
input_cost = input_tokens * (0.15 / 1_000_000)

# Print each line
print(f"Thought tokens: {thought_tokens}  Cost: ${thought_cost:.6f}")
print(f"Output tokens: {output_tokens}  Cost: ${output_cost:.6f}")
print(f"Input tokens: {input_tokens}  Cost: ${input_cost:.6f}")

# Print total cost
total_cost = thought_cost + output_cost + input_cost
print(f"Total cost: ${total_cost:.6f}")